In [1]:
import requests
import sys
import pandas as pd
import gzip
import shutil
from datetime import date, datetime, timedelta
import time
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email import encoders
import os
import glob
import logging

base_url = os.environ["CD2_base_url"]
client_id = os.environ['CD2_client_id']
client_secret = os.environ['CD2_client_secret']
idag = date.today().isoformat()
igår = (date.today() - timedelta(days=1)).isoformat()


FSbrukar = os.environ['FSbrukar']
FSpassord = os.environ['FSpassord']

parametreCanvas = {'per_page': '100'}
hodeCanvas = {'Authorization': 'Bearer ' + os.environ['tokenCanvas']}


def send_epost(tittel, innhald, avsender, mottakarar, vedlegg):
    msg = MIMEMultipart()
    msg['Subject'] = tittel
    msg['From'] = avsender
    msg['To'] = ', '.join(mottakarar)
    
    # Attach the email body to the message
    if vedlegg != "":
        msg.attach(MIMEText(innhald, 'plain'))
    
    try:
        with open(vedlegg, "rb") as attachment:
            # Create the attachment
            part = MIMEBase('application', 'octet-stream')
            part.set_payload(attachment.read())
            encoders.encode_base64(part)
            part.add_header('Content-Disposition', f'attachment; filename= {vedlegg}')
            
            # Attach the file to the email
            msg.attach(part)
        
        # Set up the server and send the email
        smtp_server = smtplib.SMTP('smtp-ut.hvl.no', port=25)
        smtp_server.sendmail(avsender, mottakarar, msg.as_string())
        smtp_server.quit() 
        logger.info(f"Sendte e-post til {', '.join(mottakarar)}")

    except Exception as e:
        logger.error(f"Feil ved sending av e-post: {e}")

def query_FS_graphql(query, variable):
    hode = {
        'Accept': 'application/json;version=1',
        "Feature-Flags": "beta, experimental"
    }
    GraphQLurl = "https://api.fellesstudentsystem.no/graphql/"
    svar = requests.post(
        GraphQLurl, 
        json = {
            'query': query,
            'variables': variable
        },
        headers=hode,
        auth=(FSbrukar, FSpassord))
    if 200 <= svar.status_code < 300:
        return svar.json()
    else:
        return {}


# opprett ein logger
logger = logging.getLogger('my_logger')
logger.setLevel(logging.DEBUG)  # Sett ønska loggnivå

# Opprett formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Opprett filhandler for å logge til fil (ein loggfil kvar dag)
file_handler = logging.FileHandler(f'loggfil-web_log-{idag}.log')
file_handler.setLevel(logging.DEBUG)
file_handler.setFormatter(formatter)

# Opprett konsollhandler for å logge til konsollen
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.DEBUG)
console_handler.setFormatter(formatter)

# Legg til handlerne i loggeren
logger.addHandler(file_handler)
logger.addHandler(console_handler)

query = """
    query MyQuery($emnekode: [String!]) {
    emner(
        filter: {eierInstitusjonsnummer: "203", emnekoder: $emnekode, ikkeUtloptITermin: {arstall: 2025, terminbetegnelse: VAR}}
    )  {
    nodes {
      personroller(filter: {erAktiv: true}) {
        fsRolle {
          navn {
            publiseringsnavn
          }
          erAktiv
        }
        fagperson {
          navn {
            etternavn
            fornavn
          }
          feideBruker
        }
        emne {
          navnAlleSprak {
            nn
          }
        }
      }
    }
  }
}
"""


In [8]:
start_les_sider = time.perf_counter()
sider = pd.read_csv("sider.csv")
sider.info()

# Finn og send melding om mest sette sider
avsendar = "aasmund.kvamme@hvl.no"
mottakarar = ["aasmund.kvamme@hvl.no"]
# mottakarar = ["aasmund.kvamme@hvl.no", "alisa.rysaeva@hvl.no", "rdeb@hvl.no"]
tittel = "Mest sette sider"
# innhald = f"Dei ti mest sette sidene {igår}\n"
# innhald += f"Antal\tSide\n"
# for i in range(10):
#     innhald += f"{sider.iloc[i]['frekvens']}\thttps://hvl.instructure.com{sider.iloc[i]['index']}\n"
# innhald += "\nLitt statistikk:\n"
# innhald += f"Antal unike sider besøkt: {len(sider)}\n"
# innhald += f"Antal sider besøkt ein gang: {len(sider[sider['frekvens']==1])}"

# innhald += "\n\n"

innhald += f"Dei ti mest sette emna {igår}\n"
# mest_sett_emne = sider['emne'].value_counts()
mest_sett = sider.groupby('emne').sum().sort_values('frekvens', ascending=False)
for i, r in sider[sider['emne']==mest_sett.index[0]].iterrows():
    print(f"{r['frekvens']} ganger:\t{r['index']}")
for s in sider['emne'].value_counts().index[0:2]:
    canvasurl = f"https://hvl.instructure.com/api/v1/courses/{s}"
    responsCanvas = requests.get(canvasurl, headers=hodeCanvas, params=parametreCanvas)
    if 200 <= responsCanvas.status_code < 300:
        dataCanvas = responsCanvas.json()
        if dataCanvas['sis_course_id'][0:1] == "U":
            emnekode = dataCanvas['sis_course_id'].split('_')[2]
            variable = {"emnekode": emnekode}
            svar = query_FS_graphql(query, variable)
            personroller = svar['data']['emner']['nodes'][0]['personroller']
            namneliste = []
            for p in personroller:
                namn = p['fagperson']['navn']['fornavn'] + ' ' + p['fagperson']['navn']['etternavn']
                brukarnamn = p['fagperson']['feideBruker']
                namneliste.append([namn, brukarnamn])
            emnenamn = svar['data']['emner']['nodes'][0]['personroller'][0]['emne']['navnAlleSprak']['nn']
            if len(namneliste) == 1:
                innhald += f"{sider[sider['emne']==str(s)].count()['frekvens']}\thttps://hvl.instructure.com/courses/{s} {emnekode} {emnenamn}. Emneansvarleg: {namneliste[0][0]}, {namneliste[0][1]}\n"
            else:
                innhald += f"{sider[sider['emne']==str(s)].count()['frekvens']}\thttps://hvl.instructure.com/courses/{s} {emnekode} {emnenamn}. Emneansvarlege: "; end=""
                for p in namneliste:
                    innhald += f"{p[0]} ({p[1]}), "
                innhald += "\n"
tidsbruk_les_sider = time.perf_counter() - start_les_sider
innhald += f"\nSamla tidsbruk: {tidsbruk_les_sider:.2f} sekund"

vedlegg = "sider.csv"


print(innhald)
# send_epost(tittel, innhald, avsendar, mottakarar, vedlegg)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1618 entries, 0 to 1617
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     1618 non-null   object
 1   frekvens  1618 non-null   int64 
 2   emne      1618 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 38.1+ KB
27 ganger:	/courses/30757/pages/terms-slash-themes-for-indigenous-young-adult-novels
23 ganger:	/courses/30757/pages/terms-slash-themes-for-indigenous-young-adult-novels?module_item_id=921315
22 ganger:	/courses/30757/pages/course-schedule?module_item_id=876722
20 ganger:	/courses/30757/pages
14 ganger:	/courses/30757/wiki
9 ganger:	/courses/30757/pages/apple-skin-to-the-core-introductory-questions?module_item_id=921324
8 ganger:	/courses/30757/pages/terms-slash-themes-for-indigenous-young-adult-novels/edit
7 ganger:	/courses/30757/pages/literary-terms-for-poetry?module_item_id=921339
5 ganger:	/courses/30757/pages/apple-skin-to-the-core-introdu

In [3]:
namneliste

[['Harald Spångberg', 'hsp@hvl.no'],
 ['Erik Andreas Hanson', 'eaha@hvl.no'],
 ['Camilla Fluge Bortheim', 'cfbo@hvl.no']]

In [4]:
namneliste[0][0]

'Harald Spångberg'

In [9]:
sider

,index,frekvens,emne
0,/courses/29421/pages/transkribering-elaeringsm...,42,29421
1,/courses/30968/pages/object-relational-mapping...,32,30968
2,/courses/30968/pages/jpa-slash-orm,31,30968
3,/courses/30225/pages/sporsmal-og-svar?module_i...,30,30225
4,/courses/28654/pages/livssynsmangfold-og-mangf...,28,28654
...,...,...,...
1613,/courses/30149/pages/case-og-kunnskapssporsmal,1,30149
1614,/courses/29110/pages/march-21st-teaching-liter...,1,29110
1615,/courses/28123/pages/uke-10-kap-6-dot-1-6-dot-...,1,28123
1616,/courses/29256/pages/uke-12-18-22-dot-mars?mod...,1,29256


In [12]:
sider[sider['index']]

KeyError: "None of [Index(['/courses/29421/pages/transkribering-elaeringsmodul?module_item_id=881462',\n       '/courses/30968/pages/object-relational-mapping-med-jakarta-persistence-api',\n       '/courses/30968/pages/jpa-slash-orm',\n       '/courses/30225/pages/sporsmal-og-svar?module_item_id=870280',\n       '/courses/28654/pages/livssynsmangfold-og-mangfoldskompetanse?module_item_id=889238',\n       '/courses/30757/pages/terms-slash-themes-for-indigenous-young-adult-novels',\n       '/courses/30553/pages/uke-12-kap-7-skjaerspenninger-torsjon?module_item_id=920478',\n       '/courses/29012/pages/anbefalt-litteratur-modul-2-ogsa-relevant-for-dei-andre-modulane?module_item_id=873622',\n       '/courses/30757/pages/terms-slash-themes-for-indigenous-young-adult-novels?module_item_id=921315',\n       '/courses/30757/pages/course-schedule?module_item_id=876722',\n       ...\n       '/courses/28283/pages/tidligere-eksamensoppgaver',\n       '/courses/30771/pages/uke-13', '/courses/30771/p

In [13]:
sider

,index,frekvens,emne
0,/courses/29421/pages/transkribering-elaeringsm...,42,29421
1,/courses/30968/pages/object-relational-mapping...,32,30968
2,/courses/30968/pages/jpa-slash-orm,31,30968
3,/courses/30225/pages/sporsmal-og-svar?module_i...,30,30225
4,/courses/28654/pages/livssynsmangfold-og-mangf...,28,28654
...,...,...,...
1613,/courses/30149/pages/case-og-kunnskapssporsmal,1,30149
1614,/courses/29110/pages/march-21st-teaching-liter...,1,29110
1615,/courses/28123/pages/uke-10-kap-6-dot-1-6-dot-...,1,28123
1616,/courses/29256/pages/uke-12-18-22-dot-mars?mod...,1,29256


In [20]:
for s in sider['index']:
    if "?" in s:
        print(s)

/courses/29421/pages/transkribering-elaeringsmodul?module_item_id=881462
/courses/30225/pages/sporsmal-og-svar?module_item_id=870280
/courses/28654/pages/livssynsmangfold-og-mangfoldskompetanse?module_item_id=889238
/courses/30553/pages/uke-12-kap-7-skjaerspenninger-torsjon?module_item_id=920478
/courses/29012/pages/anbefalt-litteratur-modul-2-ogsa-relevant-for-dei-andre-modulane?module_item_id=873622
/courses/30757/pages/terms-slash-themes-for-indigenous-young-adult-novels?module_item_id=921315
/courses/30757/pages/course-schedule?module_item_id=876722
/courses/28205/pages/pamelding-til-uteskole-hordamuseet-fredag-25-april?module_item_id=920888
/courses/28071/pages/tannhjul?module_item_id=919093
/courses/29012/pages/anbefalt-litteratur-modul-1-ogsa-relevant-for-dei-andre-modulane?module_item_id=873621
/courses/29295/pages/eksamen-mamet1kf?module_item_id=879117
/courses/28170/pages/ukeplan-12?module_item_id=918426
/courses/30759/pages/syntactic-analysis?module_item_id=901191
/courses/3

In [23]:
sider[sider['index'].str.contains("module_item_id=")]

,index,frekvens,emne
0,/courses/29421/pages/transkribering-elaeringsm...,42,29421
3,/courses/30225/pages/sporsmal-og-svar?module_i...,30,30225
4,/courses/28654/pages/livssynsmangfold-og-mangf...,28,28654
6,/courses/30553/pages/uke-12-kap-7-skjaerspenni...,26,30553
7,/courses/29012/pages/anbefalt-litteratur-modul...,23,29012
...,...,...,...
1603,/courses/27989/pages/21-mars-litteraturformidl...,1,27989
1607,/courses/28705/pages/framdriftsplan-bergen-og-...,1,28705
1611,/courses/28192/pages/referansegruppe?module_it...,1,28192
1616,/courses/29256/pages/uke-12-18-22-dot-mars?mod...,1,29256


In [25]:
sider[sider['emne']==30225]

,index,frekvens,emne
3,/courses/30225/pages/sporsmal-og-svar?module_i...,30,30225
1533,/courses/30225/wiki,1,30225
1534,/courses/30225/pages,1,30225
